In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [2]:
import IPython.display as ipd
from ponysynth.corpus import *

luna = SpeechCorpus('/home/celestia/data/audio-tar/Luna.tar')

In [3]:
luna.build_phone_index()

In [ ]:
from ponysynth.models.triphones import *
from itertools import chain

def timeit(tag, f):
    start = time.time()
    result = f()
    end = time.time()
    print('{}: {}'.format(tag, end - start))
    return result

short_set = list(dataset(luna, 5)['generator']())
short_set = chain.from_iterable(zip(short_set[:5], short_set[5:]))

for sample in short_set:
    ipd.display(ipd.Audio(sample['sample'], rate=48000))

In [28]:
def timeit(tag, f):
    start = time.time()
    result = f()
    end = time.time()
    print('{}: {}'.format(tag, end - start))
    return result

from ponysynth.models.triphones import *

raw_data = dataset(luna, size=20)
features = preprocess(**raw_data)
tfds_data = tf.data.Dataset.from_generator(**features)
tfds_data = tfds_data.prefetch(16)

def fun():
    for elem in tfds_data:
        pass
    print(elem['sample.pcm'].shape)
    print(elem['volume.rate'])
    print(elem['volume.decibals'])
    ipd.display(ipd.Audio(elem['sample.pcm'].numpy(), rate=elem['sample.rate'].numpy()))


    
timeit('fun', lambda: fun())

(8880,)
tf.Tensor(750, shape=(), dtype=int32)
tf.Tensor(
[ -7.7807727  -7.7132397  -7.5450835  -7.2624574  -6.8602967  -6.369185
  -5.965342   -5.735151   -5.5037246  -5.2537565  -5.067545   -4.8659124
  -4.600003   -4.320516   -4.0635424  -3.8417428  -3.646038   -3.4752476
  -3.3081825  -3.1854537  -3.1455266  -3.1364737  -3.1019163  -3.0338216
  -2.9251513  -2.8115087  -2.7282715  -2.6490467  -2.5577629  -2.4503236
  -2.3470657  -2.279527   -2.2354157  -2.2341955  -2.3015618  -2.395532
  -2.5095587  -2.6611495  -2.8382409  -3.0564291  -3.312278   -3.5817976
  -3.8819718  -4.1820107  -4.422889   -4.6019883  -4.727359   -4.7957945
  -4.812118   -4.7851467  -4.726037   -4.638403   -4.528238   -4.4043717
  -4.2689614  -4.1293964  -3.9946544  -3.8639584  -3.7412398  -3.632375
  -3.5378895  -3.4603777  -3.402071   -3.35822    -3.324996   -3.299771
  -3.2781134  -3.2567449  -3.2299187  -3.1889496  -3.1284359  -3.0322528
  -2.890274   -2.7090003  -2.4930987  -2.3145683  -2.3281925  -2.365346

fun: 0.9940242767333984


In [ ]:
from ponysynth.corpus import *
from ponysynth.models.triphones import *
from ponysynth.google_crud import dynamic_unroll
import librosa.display
import matplotlib.pyplot as plt

luna = SpeechCorpus('/home/celestia/data/audio-tar/Luna.tar')
luna.build_phone_index()

raw_data = dataset(luna, size=20)
features = preprocess(**raw_data)
tfds_data = tf.data.Dataset.from_generator(**features)
tfds_data = tfds_data.prefetch(16)




def run():
    for d in tfds_data.batch(1):
        print(d['log_power'])
        #outputs = model(d)
        ipd.display(ipd.Audio(d['pcm'], rate=48000))
        display_mels(d['log_mel_spectrogram'][0])
        print(d['gci'][0])
        


timeit('run', run)

In [29]:
# need 4 networks
# 1: given volume (amplitude), find out where the diphones were stitched together
#  also find the clip whose volume was modified
# 2: given pacing (gci), find out where the diphones were stitched together
#  also find the clip whose pacing was modified
# 3: given pitch (mean, variance of frequency), find out where the diphones were stitched together
#  also find the clip whose pitch was modified
# 4: given the spectrogram of a fake triphone, find out where the diphones were stitched together
#  also find the clip that was modified

import matplotlib as plt

<function sonnet.src.once.once(f)>

In [5]:
from ponysynth.corpus import *
from ponysynth.models.triphones import dataset, preprocess
import tensorflow as tf
import tensorflow_probability as tfp
import sonnet as snt
import time
# TODO: submit a pull request to deepmind/sonnet for the ConvNDTranspose issue
from ponysynth.conv_transpose import Conv1DTranspose

datapath = '/home/celestia/data'

tfd = tfp.distributions

def timeit(tag, f):
  start = time.time()
  result = f()
  end = time.time()
  print('{}: {}'.format(tag, end - start))
  return result

luna = SpeechCorpus('{}/audio-tar/Luna.tar'.format(datapath))
luna.build_phone_index()

raw_data = dataset(luna, size=20)
features = preprocess(**raw_data)
tfds_data = tf.data.Dataset.from_generator(**features)
#tfds_data = tfds_data.shuffle()
tfds_data = tfds_data.prefetch(16)


# create an embedding that captures local information
# classify embeddings as "continuous" or "contains discontinuity"

class SeqEncoder(snt.Module):
  def __init__(self, latent_dims=2):
    snt.Module.__init__(self, None)
    self.latent_dims = latent_dims

    self.prior = tfd.MultivariateNormalDiag(
      loc=tf.zeros([self.latent_dims]),
      scale_diag=tf.ones([self.latent_dims]))

  @snt.once
  def _initialize(self, inputs):
    self.mean_kernel = snt.Conv1D(
      output_channels=self.latent_dims,
      kernel_shape=3,
      padding='SAME')
    self.stddev_kernel = snt.Conv1D(
      output_channels=self.latent_dims,
      kernel_shape=3,
      padding='SAME')

      

  def __call__(self, inputs):
    self._initialize(inputs)
    means = self.mean_kernel(inputs)
    stddevs = self.stddev_kernel(inputs)

    return tfd.MultivariateNormalDiag(
      loc=means, scale_diag=stddevs**2)


class SeqDecoder(snt.Module):
  def __init__(self):
    snt.Module.__init__(self, None)

  @snt.once
  def _initialize(self, inputs):
    self.mean_kernel = Conv1DTranspose(
      output_channels=1,
      kernel_shape=3,
      padding='SAME')
    self.stddev_kernel = Conv1DTranspose(
      output_channels=1,
      kernel_shape=3,
      padding='SAME')

  def __call__(self, inputs):
    self._initialize(inputs)
    means = self.mean_kernel(inputs)
    stddevs = self.stddev_kernel(inputs)

    return tfd.MultivariateNormalDiag(
      loc=means, scale_diag=stddevs**2)


def estim_spec(features, labels, mode, params, config):
  encoder = SeqEncoder()
  decoder = SeqDecoder()

  latent_prior = encoder.prior
  latent_posterior = encoder(features)
    

class ContinuityClassifier(snt.Module):
  pass


class VolDiscriminator(snt.Module):
  pass


class PitchDiscriminator(snt.Module):
  pass


class PaceDiscriminator(snt.Module):
  pass


def _main():
  encoder = SeqEncoder()
  decoder = SeqDecoder()

  for example in tfds_data.batch(1):
    pitches = example['pitch.mean']
    pitches = tf.reshape(pitches, shape=(*pitches.shape, 1))

    print(pitches.shape)

    # shape: (batchSize, frames, channels)
    enc_samples = encoder(pitches).sample(1)[0]
    dec_samples = decoder(enc_samples).sample(1)[0]

    print(dec_samples.shape)


timeit('main', _main())

(1, 72, 1)
(1, 72, 1)
(1, 50, 1)
(1, 50, 1)
(1, 56, 1)
(1, 56, 1)
(1, 28, 1)
(1, 28, 1)
(1, 52, 1)
(1, 52, 1)
(1, 68, 1)
(1, 68, 1)
(1, 37, 1)
(1, 37, 1)
(1, 41, 1)
(1, 41, 1)
(1, 50, 1)
(1, 50, 1)
(1, 53, 1)
(1, 53, 1)
(1, 66, 1)
(1, 66, 1)
(1, 67, 1)
(1, 67, 1)
(1, 48, 1)
(1, 48, 1)
(1, 35, 1)
(1, 35, 1)
(1, 70, 1)
(1, 70, 1)
(1, 86, 1)
(1, 86, 1)
(1, 90, 1)
(1, 90, 1)
(1, 26, 1)
(1, 26, 1)
(1, 56, 1)
(1, 56, 1)
(1, 54, 1)
(1, 54, 1)


TypeError: 'NoneType' object is not callable